# Importing the Required Packages

In [1]:
import numpy as np
import timeit
import cython

In [2]:
%load_ext Cython

____

# Factorial

Below are various different but equivalent ways to implement a factorial in Python.

In [3]:
# compute x factorial using recursion
def factorial_recursive(x):
    if x == 0:
        return 1
    return x * factorial_recursive(x - 1)

# compute x factorial using a for-loop
def factorial_for(x):
    prod = 1
    for i in range(1, x + 1):
        prod *= i
    return prod

# compute x factorial using a while-loop
def factorial_while(x):
    prod = 1
    while x > 0:
        prod *= x
        x -= 1
    return prod

## Timing

In [4]:
num = 10

In [5]:
print(factorial_recursive(num))
%timeit factorial_recursive(num)

3628800
2.67 µs ± 360 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [7]:
print(factorial_for(num))
%timeit factorial_for(num)

3628800
908 ns ± 39.6 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [6]:
print(factorial_while(num))
%timeit factorial_while(num)

3628800
706 ns ± 36.4 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [8]:
import math
print(math.factorial(num))
%timeit math.factorial(num)

3628800
169 ns ± 11.2 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [9]:
print(np.math.factorial(num))
%timeit np.math.factorial(num)

3628800
194 ns ± 12 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Compiled tests

### No optimizations

In [10]:
%%cython --annotate

# compute x factorial using recursion
def factorial_recursive_unopt(x):
    if x == 0:
        return 1
    return x * factorial_recursive_unopt(x - 1)

# compute x factorial using a for-loop
def factorial_for_unopt(x):
    prod = 1
    for i in range(1, x + 1):
        prod *= i
    return prod

# compute x factorial using a while-loop
def factorial_while_unopt(x):
    prod = 1
    while x > 0:
        prod *= x
        x -= 1
    return prod

/home/pika/anaconda3/compiler_compat/ld: /lib64/libc.so.6: unknown type [0x13] section `.relr.dyn'
/home/pika/anaconda3/compiler_compat/ld: skipping incompatible /lib64/libc.so.6 when searching for /lib64/libc.so.6
/home/pika/anaconda3/compiler_compat/ld: cannot find /lib64/libc.so.6
/home/pika/anaconda3/compiler_compat/ld: /lib64/libc.so.6: unknown type [0x13] section `.relr.dyn'
/home/pika/anaconda3/compiler_compat/ld: skipping incompatible /lib64/libc.so.6 when searching for /lib64/libc.so.6
/home/pika/anaconda3/compiler_compat/ld: /lib64/ld-linux-x86-64.so.2: unknown type [0x13] section `.relr.dyn'
/home/pika/anaconda3/compiler_compat/ld: skipping incompatible /lib64/ld-linux-x86-64.so.2 when searching for /lib64/ld-linux-x86-64.so.2
/home/pika/anaconda3/compiler_compat/ld: cannot find /lib64/ld-linux-x86-64.so.2
/home/pika/anaconda3/compiler_compat/ld: /lib64/ld-linux-x86-64.so.2: unknown type [0x13] section `.relr.dyn'
/home/pika/anaconda3/compiler_compat/ld: skipping incompatibl

LinkError: command '/usr/bin/gcc' failed with exit code 1

In [11]:
num = 10

In [12]:
print(factorial_recursive_unopt(num))
%timeit factorial_recursive_unopt(num)

3628800
326 ns ± 30 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [13]:
print(factorial_for_unopt(num))
%timeit factorial_for_unopt(num)

3628800
393 ns ± 33.7 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [14]:
print(factorial_while_unopt(num))
%timeit factorial_while_unopt(num)

3628800
314 ns ± 3.34 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


#### Observations
We can see that we get a 3x performance gain by only executing cython without any optimizaions.

### With optimizations

In [16]:
%%cython --annotate

# we need to define the function in C
# recursively calling functions in Cython requires a lot of Python
# interaction
cdef int factorial_c(int x):
    if x == 0:
        return 1
    return x * factorial_c(x - 1)

# compute x factorial using recursion
def factorial_recursive_opt(int x):
    return factorial_c(x)

# compute x factorial using a for-loop
def factorial_for_opt(int x):
    cdef int prod = 1
    cdef int i
    for i in range(1, x + 1):
        prod *= i
    return prod

# compute x factorial using a while-loop
def factorial_while_opt(int x):
    cdef int prod = 1
    while x > 0:
        prod *= x
        x -= 1
    return prod

In [17]:
num = 10

In [18]:
print(factorial_recursive_opt(num))
%timeit factorial_recursive_opt(num)

3628800
57.9 ns ± 2.03 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [19]:
print(factorial_for_opt(num))
%timeit factorial_for_opt(num)

3628800
57.3 ns ± 2.29 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [20]:
print(factorial_while_opt(num))
%timeit factorial_while_opt(num)

3628800
49.9 ns ± 3.13 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


#### Observations
With optimizations, by annotating types properly, we can improve the performance by 10x, which is faster than the built-in functions.

____

# Gaussian elimination

The below code performs Gaussian elimination with partial pivoting.

In [22]:
# solves Ax = b and returns x
def solve(A, b):
    n = A.shape[0] # number of unknowns
    B = np.concatenate((A, np.expand_dims(b, axis=1)), axis=1) # augmented matrix
    
    # bring B to row echelon form
    for i in range(0, n): # loop through diagonal elements
        # implement partial pivoting
        # find the maximum absolute value in this column
        max_k = i
        for k in range(i + 1, n):
            if np.abs(B[k][i]) > np.abs(B[max_k][i]):
                max_k = k
                
        if B[max_k][i] == 0:
            raise ZeroDivisionError("unsolvable matrix")
                
        # swap rows so that the maximum value is our new pivot
        B[[i, max_k]] = B[[max_k, i]]
        
        # reduce the values below
        for j in range(i + 1, n): # loop through rows below
            B[j,i:n+1] -= (B[j][i] / B[i][i]) * B[i,i:n+1]
        
    # now find the variables
    for i in range(n - 1, -1, -1): # loop up the matrix
        B[i][n] /= B[i][i]
        B[0:i,n] -= B[i][n] * B[0:i,i]
        
    return B[0:n,n]

Now we can perform a benchmark:

In [23]:
A = np.random.rand(10, 10).astype('complex128')
b = np.random.rand(10).astype('complex128')

In [24]:
print(np.real(solve(A, b)))
%timeit solve(A, b)

[ 0.33937697 -0.05591564  0.35026955  0.82099175 -0.03659062  0.11177819
 -0.31301781 -0.55932231  0.54992376 -0.11725079]
514 µs ± 25.4 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


Comparing with the numpy function:

In [25]:
print(np.real(np.linalg.solve(A, b)))
%timeit np.linalg.solve(A, b)

[ 0.33937697 -0.05591564  0.35026955  0.82099175 -0.03659062  0.11177819
 -0.31301781 -0.55932231  0.54992376 -0.11725079]
25.5 µs ± 1.14 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Cython

We first just run with simple compilation using Cython.

### Without any changes made

In [27]:
%%cython --annotate

import numpy as np

# solves Ax = b and returns x
def solve_1(A, b):
    n = A.shape[0] # number of unknowns
    B = np.concatenate((A, np.expand_dims(b, axis=1)), axis=1) # augmented matrix
    
    # bring B to row echelon form
    for i in range(0, n): # loop through diagonal elements
        # implement partial pivoting
        # find the maximum absolute value in this column
        max_k = i
        for k in range(i + 1, n):
            if np.abs(B[k][i]) > np.abs(B[max_k][i]):
                max_k = k
                
        if B[max_k][i] == 0:
            raise ZeroDivisionError("unsolvable matrix")
                
        # swap rows so that the maximum value is our new pivot
        B[[i, max_k]] = B[[max_k, i]]
        
        # reduce the values below
        for j in range(i + 1, n): # loop through rows below
            B[j,i:n+1] -= (B[j][i] / B[i][i]) * B[i,i:n+1]
        
    # now find the variables
    for i in range(n - 1, -1, -1): # loop up the matrix
        B[i][n] /= B[i][i]
        B[0:i,n] -= B[i][n] * B[0:i,i]
        
    return B[0:n,n]

In [28]:
print(np.real(solve_1(A, b)))
%timeit solve_1(A, b)

[ 0.33937697 -0.05591564  0.35026955  0.82099175 -0.03659062  0.11177819
 -0.31301781 -0.55932231  0.54992376 -0.11725079]
533 µs ± 29.2 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


As we can observe, since there is a very large amount of Python interaction, there is no performance gain by just running Cython on the code.

### Witih basic type annotation

In [29]:
%%cython --annotate

import numpy as np

# solves Ax = b and returns x
def solve_2(double complex[:, :] A, double complex[:] b):
    cdef Py_ssize_t n = A.shape[0] # number of unknowns
    
    cdef Py_ssize_t i, j
    
    # augmented matrix
    cdef double complex[:, :] B = np.concatenate(
            (A, np.expand_dims(b, axis=1)), axis=1
        )
    
    cdef Py_ssize_t k, max_k
    cdef double complex temp, ratio
    
    # bring B to row echelon form
    for i in range(0, n): # loop through diagonal elements
        # implement partial pivoting
        # find the maximum absolute value in this column
        max_k = i
        
        for k in range(i + 1, n):
            if abs(B[k, i]) > abs(B[max_k, i]):
                max_k = k
                
        if B[max_k, i] == 0:
            raise ZeroDivisionError("unsolvable matrix")
                
        # swap rows so that the maximum value is our new pivot
        for j in range(0, n + 1):
            temp = B[i, j]
            B[i, j] = B[max_k, j]
            B[max_k, j] = temp
        
        # reduce the values below
        for j in range(i + 1, n): # loop through rows below
            ratio = B[j, i] / B[i, i]
            
            for k in range(i, n + 1):
                B[j, k] = B[j, k] - ratio * B[i, k]
        
    # now find the variables
    for i in range(n - 1, -1, -1): # loop up the matrix
        B[i, n] = B[i, n] / B[i, i]
        for j in range(0, i):
            B[j, n] = B[j, n] - B[i, n] * B[j, i]
        
    return B[0:n,n]

In [30]:
print(np.real(solve_2(A, b)))
%timeit solve_2(A, b)

[ 0.33937697 -0.05591564  0.35026955  0.82099175 -0.03659062  0.11177819
 -0.31301781 -0.55932231  0.54992376 -0.11725079]
11.4 µs ± 398 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


We can already see that by just annotating types, we have improved our performance by 50x, beating the built-in numpy code by 2x.

However, we can see that the part where we construct the augmented matrix still has a large amount of Python interaction.

### Using our own function for the augmented matrix

In [34]:
%%cython --annotate

import numpy as np

# creates an augmented matrix by putting b to the right of A
cdef double complex[:, :] augment(double complex[:, :] A, double complex[:] b):
    cdef Py_ssize_t n = A.shape[0]
    cdef double complex[:, :] B = np.zeros((n, n + 1), dtype=np.complex128)
  
    cdef Py_ssize_t i, j
  
    for i in range(0, n):
        for j in range(0, n):
            B[i, j] = A[i, j]
        B[i, n] = b[i]
      
    return B

# solves Ax = b and returns x
def solve_3(double complex[:, :] A, double complex[:] b):
    cdef Py_ssize_t n = A.shape[0] # number of unknowns
    
    cdef Py_ssize_t i, j
    
    # augmented matrix
    cdef double complex[:, :] B = augment(A, b)
    
    cdef Py_ssize_t k, max_k
    cdef double complex temp, ratio
    
    # bring B to row echelon form
    for i in range(0, n): # loop through diagonal elements
        # implement partial pivoting
        # find the maximum absolute value in this column
        max_k = i
        
        for k in range(i + 1, n):
            if abs(B[k, i]) > abs(B[max_k, i]):
                max_k = k
                
        if B[max_k, i] == 0:
            raise ZeroDivisionError("unsolvable matrix")
                
        # swap rows so that the maximum value is our new pivot
        for j in range(0, n + 1):
            temp = B[i, j]
            B[i, j] = B[max_k, j]
            B[max_k, j] = temp
        
        # reduce the values below
        for j in range(i + 1, n): # loop through rows below
            ratio = B[j, i] / B[i, i]
            
            for k in range(i, n + 1):
                B[j, k] = B[j, k] - ratio * B[i, k]
        
    # now find the variables
    for i in range(n - 1, -1, -1): # loop up the matrix
        B[i, n] = B[i, n] / B[i, i]
        for j in range(0, i):
            B[j, n] = B[j, n] - B[i, n] * B[j, i]
        
    return B[0:n,n]

In [35]:
print(np.real(solve_3(A, b)))
%timeit solve_3(A, b)

[ 0.33937697 -0.05591564  0.35026955  0.82099175 -0.03659062  0.11177819
 -0.31301781 -0.55932231  0.54992376 -0.11725079]
3.57 µs ± 240 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


We have improved our performance by a further 3x by removing this bottleneck.

### Timing tests

In [150]:
print(np.real(solve(A, b)))
%timeit solve(A, b)

[ 3.24554399 -4.58366764  1.07117629  2.31410306 -0.03393728 -6.36197302
 -6.87525314  5.11972583 -1.80032388  3.96545411]
498 µs ± 18.7 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [151]:
print(np.real(solve_opt(A, b)))
%timeit solve_opt(A, b)

[ 3.24554399 -4.58366764  1.07117629  2.31410306 -0.03393728 -6.36197302
 -6.87525314  5.11972583 -1.80032388  3.96545411]
3.41 µs ± 207 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [152]:
print(np.real(np.linalg.solve(A, b)))
%timeit np.linalg.solve(A, b)

[ 3.24554399 -4.58366764  1.07117629  2.31410306 -0.03393728 -6.36197302
 -6.87525314  5.11972583 -1.80032388  3.96545411]
26.7 µs ± 1.8 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Observations
As we can see, properly optimized Cython code can be much faster than any of the Python alternatives. Our optimized Gaussian elimination code is almost 9x faster than the numpy one.

# SPICE

The below code blocks set up a `SpiceSolver` class which can be used to load and solve SPICE netlists.

## Spice Solver class

In [30]:
%%cython --annotate

# an edge in the network that goes between two nodes
cdef class Edge:
    cdef string name
    cdef string node_left
    cdef string node_right
    
    cpdef def __init__(
        self, string name, string node_left, string node_right
    ):
        self.name = name
        self.node_left = node_left
        self.node_right = node_right
        
# a general passive component
cdef class Passive(Edge):
    cdef float value
    cdef float cond
    
    cpdef def __init__(
        self,
        string name,
        string node_left,
        string node_right,
        float value,
        float cond=0
    ):
        Edge.__init__(self, name, node_left, node_right)
        self.value = value
        self.cond = cond # condition, like voltage of a capacitor

# a general source (voltage or current)
cdef class Source(Edge):
    cdef float value
    cdef float phase
    
    def __init__(
        self,
        string name,
        string node_left,
        string node_right,
        float value,
        float phase=0
    ):
        Edge.__init__(self, name, node_left, node_right)
        self.value = value
        self.phase = phase

# class that parses a spice file and solves it
class SpiceSolver:
    # throw an error specifying the line where the error occurred
    @staticmethod
    cdef void __spice_err(
        Py_ssize_t line_index,
        Py_ssize_t line,
        string message
    ):
        raise Exception(
            f"SPICE ERROR on line {line_index + 1}:\n" +
            f"{line}\n" +
            message
        )
    
    # parse a float, and throw an error if it's not valid
    @staticmethod
    cdef float __parse_float(
        string x,
        Py_ssize_t line_index,
        Py_ssize_t line
    ):
        cdef float xf
        try:
            xf = float(x)
        except ValueError:
            SpiceSolver.__spice_err(line_index, line, f"couldn't parse as number: `{command[3]}`")
        return xf
    
    # validate the number of arguments to a command
    @staticmethod
    def __assert_arg_count(
        string[] command,
        Py_ssize_t[] expected,
        Py_ssize_t line_index,
        Py_ssize_t line):
        
        if len(command) not in expected:
            SpiceSolver.__spice_err(
                line_index,
                line,
                f"invalid number of arguments: expected {expected}, got {len(command)}"
            )
    
    def __init__(self):
        # we will replace all nodes with numbers in our matrix
        # this map will remember the mapping between node names and the numbering we assign
        self.node_map = { "GND": 0 }
        
        # number of nodes (excluding ground)
        self.node_count = 0
        
        # lists of sources by frequency
        self.voltages = { 0: [] }
        self.currents = { 0: [] }
        
        # store a list of components
        self.resistors = []
        self.capacitors = []
        self.inductors = []
        
        # we represent the passive elements in the circuit as 3 weighted graphs where
        # components form edges:
        # one has conductances as its weights
        # one has capacitance as its weights
        # one has 1/inductance as its weights
        
        # we use the diagonal elements as the negative of the sum of the rest of the
        # values in the corresponding row (plus ground), since this is what appears
        # in the MNA matrix
        
        # adjacency matrix of conductances between nodes (excluding GND)
        self.conductance_matrix = None
        
        # adjacency matrix of capacitances between nodes (excluding GND)
        self.capacitance_matrix = None
        
        # adjacency matrix of 1/inductances between nodes (excluding GND)
        self.inv_inductance_matrix = None
    
    # read a file and create a representation of the circuit in memory
    def read_file(self, filename):
        # open file
        with open(filename) as f:
            l = f.readlines()
            
            if len(l) == 0: # make sure the file is not empty
                raise Exception("SPICE ERROR: empty file")

            # seek to start of circuit
            start = 0
            while len(l[start]) > 0 and l[start].split("#")[0].strip() != ".circuit":
                start += 1
                if start >= len(l): # we need to have a ".circuit" somewhere
                    raise Exception("SPICE ERROR: couldn't find start of circuit")
                    
            # find end of circuit
            end = start
            while len(l[end]) > 0 and l[end].split("#")[0].strip() != ".end":
                end += 1
                if end >= len(l): # we need to have a ".end" somewhere
                    raise Exception("SPICE ERROR: couldn't find end of circuit")
            
            # generate a dictionary of source names to frequencies based on the directives given after ".end"
            frequencies = {}
            
            # loop through lines after the ".end" directive
            for i in range(end + 1, len(l)):
                command = l[i].split('#')[0].split() # get command
                
                if command[0] == "": # if there's no command, move on
                    continue
                
                # look for and parse ".ac" command
                if command[0].lower() == ".ac":
                    self.__assert_arg_count(command, [3], i, l[i])
                    
                    if command[1].upper() in frequencies: # each source should have its frequency defined only once
                        self.__spice_error(i, l[i], "redefinition of freqency")
                        
                    # add frequency to dictionary
                    freq = self.__parse_float(command[2], i, l[i])
                    frequencies[command[1].upper()] = freq

                    self.voltages[freq] = []
                    self.currents[freq] = []
                else:
                    pass # we don't need to throw errors if there is garbage after the ".end"
            
            # generate a list of nodes and components, and check syntax
            for i in range(start + 1, end): # loop through the ".circuit" section
                command = l[i].split('#')[0].split()

                if command[0] == "":
                    continue
                    
                if len(command) < 4: # all commands have at least 4 arguments
                    self.__spice_error(i, l[i], "invalid number of arguments")
                
                # the second and third arguments will contain node names
                # add the nodes mentioned to our node map if they aren't already in it
                for j in [1, 2]:
                    node_name = command[j]
                    if node_name.upper() not in self.node_map:
                        self.node_count += 1
                        self.node_map[node_name.upper()] = self.node_count
                
                # type of component
                ctype = l[i][0].upper()

                # parse passive components
                if ctype in ["R", "L", "C"]:
                    self.__assert_arg_count(command, [4, 5], i, l[i])
                    
                    edge = Passive(
                        command[0],
                        self.node_map[command[1].upper()],
                        self.node_map[command[2].upper()],
                        self.__parse_float(command[3], i, l[i])
                    )
                    
                    if len(command) == 5: # if an initial condition is specified (5th argument), set it
                        edge.cond = self.__parse_float(command[4], i, l[i])
                        
                    # add value to list
                    if ctype == "R":
                        self.resistors.append(edge)
                    elif ctype == "L":
                        self.inductors.append(edge)
                    else:
                        self.capacitors.append(edge)
                        
                # parse sources
                elif ctype in ["V", "I"]:
                    freq = 0
                    phase = 0
                    
                    # check type of source
                    if command[3].lower() == "ac":
                        self.__assert_arg_count(command, [6], i, l[i])
                        
                        # look up our frequencies dictionary for this source
                        if command[0].upper() in frequencies:
                            freq = frequencies[command[0].upper()]
                            phase = self.__parse_float(command[5], i, l[i])
                        else: # throw an error if the frequency for this source is not defined
                            self.__spice_error(i, l[i], f"could not find frequency for AC source: `{command[0]}`")
            
                    elif command[3].lower() == "dc":
                        self.__assert_arg_count(command, [5], i, l[i])
                        
                    else:
                        self.__spice_error(i, l[i], f"invalid source type: `{command[3]}`")
                    
                    edge = Source(
                        command[0],
                        self.node_map[command[1].upper()],
                        self.node_map[command[2].upper()],
                        self.__parse_float(command[4], i, l[i]),
                        phase,
                    )
                    
                    if ctype == "V":
                        self.voltages[freq].append(edge)
                    else:
                        self.currents[freq].append(edge)
                        
                else:
                    self.__spice_error(i, l[i], "unidentified command inside circuit")
                    
        # delete GND
        del self.node_map["GND"]
        
        # now use the data we have parsed to generate matrices
        self.__gen_matrices()
    
    # generate conductance, capacitance and inverse-inductance matrices
    def __gen_matrices(self):
        # initialize to zeroes
        # outwards currents are taken as positive
        
        self.conductance_matrix = np.zeros((self.node_count, self.node_count))
        self.capacitance_matrix = np.zeros((self.node_count, self.node_count))
        self.inv_inductance_matrix = np.zeros((self.node_count, self.node_count))
        
        # conductance matrix
        for r in self.resistors:
            low = min(r.node_left, r.node_right) - 1
            high = max(r.node_left, r.node_right) - 1
            if low == -1:
                # if the resistor is between ground and a node, add it to the diagonal
                self.conductance_matrix[high][high] += 1 / r.value
            else:
                self.conductance_matrix[low][high] -= 1 / r.value
                
        # capacitance matrix
        for c in self.capacitors:
            low = min(c.node_left, c.node_right) - 1
            high = max(c.node_left, c.node_right) - 1
            if low == -1:
                self.capacitance_matrix[high][high] += c.value
            else:
                self.capacitance_matrix[low][high] -= c.value
                
        # inverse-inductance matrix
        for l in self.inductors:
            low = min(l.node_left, l.node_right) - 1
            high = max(l.node_left, l.node_right) - 1
            if low == -1:
                self.inv_inductance_matrix[high][high] += 1 / l.value
            else:
                self.inv_inductance_matrix[low][high] -= 1 / l.value
        
        # mirror our matrices and fix the diagonal elements
        mats = [self.conductance_matrix, self.inv_inductance_matrix, self.capacitance_matrix]
        for mat in mats:
            for i in range(self.node_count):
                # set diagonal elements
                for j in range(0, i):
                    mat[i][i] -= mat[j][i]
                    
                # mirror
                for j in range(i + 1, self.node_count):
                    mat[i][i] -= mat[i][j]
                    mat[j][i] = mat[i][j]
        
    # create a linear matrix equation A x = b for MNA (returns A and b)
    def __gen_mna_pair(self, modified_voltages, freq):
        w = 2j * np.pi * freq
        
        dim = self.node_count + len(modified_voltages) # number of dimensions in matrix
        
        mat = np.zeros((dim, dim), dtype=np.complex_)
        x = np.zeros(dim, dtype=np.complex_)
        
        
        if w != 0:
            mat[0:self.node_count, 0:self.node_count] = self.conductance_matrix + \
                                                        1j * w * self.capacitance_matrix + \
                                                        1 / (1j * w) * self.inv_inductance_matrix
        else:
            mat[0:self.node_count, 0:self.node_count] = self.conductance_matrix
            
        # outward currents are taken as positive, active sign convention is followed for voltage sources
        
        k = self.node_count
        
        # create MNA matrix
        
        # add equations of the form V_left - V_right = V_source
        # and also currents on the RHS of KCL equations
        for v in modified_voltages:
            if v.node_left != 0:
                mat[v.node_left - 1][k] = -1 # current
                mat[k][v.node_left - 1] = 1 # voltage
                
            if v.node_right != 0:
                mat[v.node_right - 1][k] = 1 # current
                mat[k][v.node_right - 1] = -1 # voltage
            
            x[k] = v.value * np.exp(1j * v.phase)
            
            k += 1
            
        for i in self.currents[freq]:
            if i.node_left != 0:
                x[i.node_left - 1] = i.value * (np.exp(1j * i.phase))
                
            if i.node_right != 0:
                x[i.node_right - 1] = -i.value * (np.exp(1j * i.phase))
            
        return (mat, x)
    
    # generate modified voltage sources according to frequency and solve,
    # returning a dictionary of nodes to voltages/currents
    def __solve_freq(self, freq):
        modified_voltages = []
        
        # map of extra nodes that we have added
        extra_nodes = {}
        
        # convert inductors to 0-voltage sources for DC
        if freq == 0:
            for l in self.inductors:
                modified_voltages.append(
                    Source(l.name, l.node_left, l.node_right, 0, 0)
                )
        
        for f in self.voltages:
            # if the source is at a different frequency, short it
            if freq != f:
                for v in self.voltages[f]:
                    modified_voltages.append(
                        Source(v.name, v.node_left, v.node_right, 0, 0)
                    )
            else:
                for v in self.voltages[f]:
                    modified_voltages.append(v)
        
        # keep track of where each voltage is
        new_node_count = self.node_count
        for v in modified_voltages:
            new_node_count += 1
            extra_nodes[f"I_{v.name}"] = new_node_count
        
        solution = solve(*self.__gen_mna_pair(modified_voltages, freq))
        
        if freq == 0:
            solution = np.real(solution)
        
        result = {}
        
        for key in self.node_map:
            result[f"V_{key}"] = solution[self.node_map[key] - 1]
            
        for key in extra_nodes:
            result[key] = solution[extra_nodes[key] - 1]
            
        return result
        
    # print the steady state solution of the system
    def solve_steady(self):
        for freq in self.voltages:
            print(f"frequency {freq}:")
            print()
            try:
                sol = self.__solve_freq(freq)
                for key in sol:
                    print(f"{key:<10}{sol[key]}")
            except ZeroDivisionError:
                print("no steady state at this frequency")

            print()
            print("=" * 20)
            print()

We can invoke the solver as follows:

It prints the different responses of the circuit to the different frequencies of sources that are present.

In [32]:
solver = SpiceSolver()
solver.read_file("spice/ckt4.netlist")
solver.solve_steady()

frequency 0:

V_1       -10.0
V_2       -5.555555555555556
V_3       -3.7037037037037037
I_V1      2.2222222222222214


